# Chain-of-thought testing - ChatGPT-4o-mini
> Liam Barrett, 19 November, 2024

The current notebook takes in the clinical letter data and retrieves predicted symptoms, diagnoses and procedures from a LLM. 

The clinical letters are openly available at [MT samples](https://mtsamples.com) and extracted by `./get_mtsamples_data.ipynb`. The letters are available under `../data/mt_samples/letters`.

This notebook aims to demo the procedure for chain-of-thought based prompting and label extraction outlined in `../docs/methods.md`

## Overview 
1. Set model as medical expert for clinical letter analysis.
2. Provide example letter and desired label format as template.
3. Extract labels from clinical letter in JSON format.
4. Review generated labels for validity and completeness.
5. Remove invalid labels and regenerate missing or modified ones.

Note, in this particular notebook, we are stepping a stage outlined in the main methods document; "Query SNOMED-CT API to get codes for each label." which would come between 3. and 4 in the above steps.

We have also deleted the step where we iterate over labels. Instead requiring the LLM to perform the how label, meta-data and qualifer extraction in one fell swoop.

**Figure 1.** LLM Clinical Letter Annotation Process Flow  

[![Figure 1. LLM Clinical Letter Annotation Process Flow](https://mermaid.ink/img/pako:eNqFk0Fv2kAQhf_KyJF6AvXAqT5UIhgIxICB9NIlh409hlXWu-56TYIw_z3jtWlM1aocLOz53szb5_HZi3WCnu-lUr_FB24sPAU7BfQbsq2l-2fo97_DPYsM9nOjs9xCGC6AF5BhImIuAd9zJK5R3Tt8RLg-igSpyLNcIki0Fg18Aa6UttwKrSDVJuNX4cgJAzZTeWkhlkK55o2uZQLHjNkUFRpuEQiyoqb4C8qCur_iCSy-W6rAfLtatsKxE05YxE2BhBX8iK4OUtMUeWq55jpx9JSFdVegk8RYFELtIdQ6b8mpYx7OE20AeXxoLFya4oMrzj59_iq5FKlAU3vM0PKEW952mjl4TmlrgxDrOi6SuH7Q4ebNxK7RZk41lL8TyBuzmFTwyNYlmhNsl6vFOOiPnmAYzdpej04Ysg0eBb5BdOAF3kQQOmBxrl91A7VHW9SFaqaOdKAEUFkjsKhgyQJ0tkVbafw8d0UL4VL8qhAT2h6dUCKx24QKVmSl3S7aCshatIHwj3YrZy5iP_KE3yT18nmMZcN059cxOXMVrNlEKNqbdhlpQC0v8LqMkVOvu4ms3aMNG6vkJqnCnmi9h5AKKf279FvaK6zRr-jfDQaDLrL5PzK9IvE_kfDviNfzMqSvSST0LZ9rwc6zB8xw5_n0N8GUl9LuvJ26EMpLq7cnFXu-NSX2PKPL_cHzUy4LuitdrIHge8OzK5Jz9VPrrIUuH5InVW8?type=png)](https://mermaid.live/edit#pako:eNqFk0Fv2kAQhf_KyJF6AvXAqT5UIhgIxICB9NIlh409hlXWu-56TYIw_z3jtWlM1aocLOz53szb5_HZi3WCnu-lUr_FB24sPAU7BfQbsq2l-2fo97_DPYsM9nOjs9xCGC6AF5BhImIuAd9zJK5R3Tt8RLg-igSpyLNcIki0Fg18Aa6UttwKrSDVJuNX4cgJAzZTeWkhlkK55o2uZQLHjNkUFRpuEQiyoqb4C8qCur_iCSy-W6rAfLtatsKxE05YxE2BhBX8iK4OUtMUeWq55jpx9JSFdVegk8RYFELtIdQ6b8mpYx7OE20AeXxoLFya4oMrzj59_iq5FKlAU3vM0PKEW952mjl4TmlrgxDrOi6SuH7Q4ebNxK7RZk41lL8TyBuzmFTwyNYlmhNsl6vFOOiPnmAYzdpej04Ysg0eBb5BdOAF3kQQOmBxrl91A7VHW9SFaqaOdKAEUFkjsKhgyQJ0tkVbafw8d0UL4VL8qhAT2h6dUCKx24QKVmSl3S7aCshatIHwj3YrZy5iP_KE3yT18nmMZcN059cxOXMVrNlEKNqbdhlpQC0v8LqMkVOvu4ms3aMNG6vkJqnCnmi9h5AKKf279FvaK6zRr-jfDQaDLrL5PzK9IvE_kfDviNfzMqSvSST0LZ9rwc6zB8xw5_n0N8GUl9LuvJ26EMpLq7cnFXu-NSX2PKPL_cHzUy4LuitdrIHge8OzK5Jz9VPrrIUuH5InVW8)

## Desired outputs
Ideally, the models should allow us to create a tabulated set of labels for a given label following the structure:

**Table 1.** Example Labels, meta-data and qualifiers for letter `0018`
| Text             	| Context                                                                                   	| Qualifier         	| Laterality 	| Presence 	| Primary/Secondary 	| Experienced 	| Treatment stage 	| SNOMED CT 	|
|------------------	|-------------------------------------------------------------------------------------------	|-------------------	|------------	|----------	|-------------------	|-------------	|-----------------	|-----------	|
| 15-year-old      	| The patient is a 15-year-old female                                                       	| Sociodemographics 	| NA         	| NA       	| NA                	| Patient     	| NA              	| 397669002 	|
| female           	| The patient is a 15-year-old female                                                       	| Sociodemographics 	| NA         	| NA       	| NA                	| Patient     	| NA              	| NA        	|
| enlarged tonsils 	| was seen in consultation at the request of Dr. X on 05/15/2008 regarding enlarged tonsils 	| Sign              	| NA         	| Suspect  	| Primary           	| Patient     	| NA              	| 275876001 	|
| tonsillitis      	| having two to three bouts of tonsillitis this year                                        	| Diagnosis         	| NA         	| Resolved 	| Primary           	| Patient     	| NA              	| 90176007  	|

Note how the `Text` column provides the exact string label ad the `Context` provides the information used to derive this label. This allows use to use the same context (rows 1 and 2) for different labels. Additionally, note how no cells in the table are empty. Rather, the cell is filled with NA if there is no supporting information for that column and that label. Finally, there are numerical labels in the form of SNOMED-CT codes in the `SNOMED CT` column. These have been manually looked up in this case but should be done programmatically in the future. The full example tabulated labelling for letter `0018` is available under `../data/letter_0018.txt` (tab delimited).

## Import libraries

In [13]:
# Import processing packages
import os
import json
import re
from typing import Dict, Optional, Any
import logging
from pathlib import Path
import uuid
from datetime import datetime
import openai
import ast
import pandas as pd
from tqdm import tqdm

# Import the openai package
import openai

# Import Notebook Packages for viewing
from IPython.display import display, Markdown, clear_output

## Define prompts

In [68]:
# Step 1: pre-prompt
PREPROMPT = '''
You are an expert medical professional with extensive experience in clinical documentation analysis. Your task is to extract and analyze information from clinical letters systematically and accurately.

Key Responsibilities:
1. Extract relevant clinical information into structured categories:
   - Socio-demographics
   - Signs
   - Symptoms
   - Diagnoses
   - Treatments
   - Risk Factors
   - Test Results

Standards for Analysis:
- Base all extractions on explicit evidence from the text
- Maintain clinical precision and accuracy
- Avoid inferring information not directly supported by the text
- Distinguish between confirmed and suspected findings
- Consider temporal relationships in the clinical narrative
- Preserve medical terminology as presented in the source text

Required Information for Each Extraction:
- Mandatory Fields:
  * Key Text: The specific phrase or term from the source text
  * Context: The complete relevant sentence or passage containing the key text
- Optional Fields (include when applicable):
  * Laterality (left, right, bilateral, NA)
  * Presence (confirmed, suspected, resolved, NA)
  * Primary/Secondary classification
  * Experiencer (patient, family member, NA)
  * Treatment stage (pre-treatment, post-treatment, NA)
  * SNOMED-CT code (if confident in the mapping)

Output Requirements:
- Maintain clinical accuracy and precision
- Present information in a structured, consistent format
- Use "NA" for any field where information is not applicable or cannot be determined from the text
- Preserve the original medical terminology

Remember:
- Only extract information explicitly present in the text
- Maintain medical accuracy and precision
- Follow a systematic approach to analysis
- Be prepared to justify each extraction with specific textual evidence
- Flag any uncertainties or ambiguities in the source text
'''

# Step 2: Provide the aims of the task and an example annotation
EXAMPLE_LETTER_PROMPT = '''
I will now show you an example clinical letter and its annotation to demonstrate the task. Your job will be to replicate this type of analysis on new clinical letters.

Example Clinical Letter (Letter 0018):
{
  "letter": 0018,
  "description": "Chronic adenotonsillitis with adenotonsillar hypertrophy. Upper respiratory tract infection with mild acute laryngitis.",
  "history": "The patient is a 15-year-old female who was seen in consultation at the request of Dr. X on 05/15/2008 regarding enlarged tonsils..."
  [rest of letter content as shown above]
}

Here is an example of how this letter should be analyzed, with each extracted label and its associated metadata in JSON format:

{
  "labels": {
    "15-year-old": {
      "context": "The patient is a 15-year-old female",
      "qualifier": "Sociodemographics",
      "laterality": "NA",
      "presence": "NA",
      "primary_secondary": "NA",
      "experiencer": "Patient",
      "treatment_stage": "NA",
      "snomed_ct": "397669002"
    },
    "female": {
      "context": "The patient is a 15-year-old female",
      "qualifier": "Sociodemographics",
      "laterality": "NA",
      "presence": "NA",
      "primary_secondary": "NA",
      "experiencer": "Patient",
      "treatment_stage": "NA",
      "snomed_ct": "NA"
    },
    "enlarged tonsils": {
      "context": "was seen in consultation at the request of Dr. X on 05/15/2008 regarding enlarged tonsils",
      "qualifier": "Sign",
      "laterality": "NA",
      "presence": "Suspect",
      "primary_secondary": "Primary",
      "experiencer": "Patient",
      "treatment_stage": "NA",
      "snomed_ct": "275876001"
    },
    "tonsillitis": {
      "context": "having two to three bouts of tonsillitis this year",
      "qualifier": "Diagnosis",
      "laterality": "NA",
      "presence": "Resolved",
      "primary_secondary": "Primary",
      "experiencer": "Patient",
      "treatment_stage": "NA",
      "snomed_ct": "90176007"
    }
  }
}

For each new clinical letter, you should:
1. Extract all relevant labels that fall into these categories:
   - Socio-demographics
   - Signs
   - Symptoms
   - Diagnoses
   - Treatments
   - Risk Factors
   - Test Results

2. For each label, provide:
   - The exact text from the letter
   - The complete context (full sentence or relevant passage)
   - Appropriate qualifier from the categories above
   - All applicable metadata fields (laterality, presence, etc.)
   - Use "NA" for any field that is not applicable or cannot be determined

3. Format your response as a JSON object following the exact structure shown in the example above.

4. Ensure that:
   - Each label is an exact quote from the text
   - Context provides sufficient information to understand the label
   - All fields are completed (using "NA" when not applicable)
   - The classification is based solely on information present in the text

Are you ready to analyze a new clinical letter following this format?
'''

# Step 3: Prompt the LLM with the letter of choice
formatted_letter = 'None' # Initialise empty letter
def create_letter_prompt(formatted_letter):
    LETTER_PROMPT_DESC = '''
Analyze the following clinical letter and extract all relevant information. Your response must be a valid JSON object matching the following specification exactly.

Required Format:
{
  "labels": {
    "exact_text": { # replace exact_text with the label. In the examples, one was "15-year-old".
      "context": "string with complete sentence/passage",
      "qualifier": "MUST BE ONE OF: Sociodemographics, Signs, Symptoms, Diagnoses, Treatments, Risk Factors, Test Results",
      "laterality": "MUST BE ONE OF: left, right, bilateral, NA",
      "presence": "MUST BE ONE OF: confirmed, suspected, resolved, negated, NA",
      "primary_secondary": "MUST BE ONE OF: primary, secondary, NA",
      "experiencer": "MUST BE ONE OF: Patient, Family, NA",
      "treatment_stage": "MUST BE ONE OF: pre-treatment, post-treatment, NA",
      "snomed_ct": "string of numbers or NA"
    }
    "exact_text": { # the rest of the labels
    [...] # the rest of the label's data etc.
    }
  }
}

Critical Requirements:
1. Response must be a single JSON object
2. Include all labels you identify
3. All field names must be exactly as shown above
4. Every label must have ALL fields specified (use "NA" if not applicable)
5. No additional fields or nested objects are allowed
6. All string values must be properly escaped
7. Fields must use ONLY the enumerated values where specified
8. Context must contain complete sentences, properly escaped
9. Do not include any explanatory text before or after the JSON

Extract all instances of:
- Socio-demographics
- Signs
- Symptoms
- Diagnoses
- Treatments
- Risk Factors
- Test Results

Clinical Letter:
'''
    LETTER_PROMPT = LETTER_PROMPT_DESC+f'''

{formatted_letter}

Respond only with the JSON object following the specified format.
'''
    return LETTER_PROMPT

# Step 4: Review the extracted JSON for errors
formatted_labels_json = 'None' # Initialise empty JSON
REVIEW_LABELS_PROMPT_INTRO = '''
Review the following extracted labels from a clinical letter. Verify their accuracy, completeness, and consistency. The current extraction is:

'''
REVIEW_LABELS_PROMPT_MAIN = '''
Review Tasks:
1. MISSING INFORMATION
   Check for any missing labels in these categories:
   - Socio-demographics
   - Signs
   - Symptoms
   - Diagnoses
   - Treatments
   - Risk Factors
   - Test Results

2. VALIDATION
   For each existing label, verify:
   - Context matches the label
   - Qualifier category is appropriate
   - Presence status is accurate
   - Primary/Secondary classification is correct
   - Experiencer assignment is accurate
   - Treatment stage is appropriate

3. CLINICAL CONSISTENCY
   Check that:
   - Related symptoms/signs are consistently labeled
   - Temporal relationships are correctly reflected
   - Family history is properly distinguished
   - Diagnostic relationships are accurate

Provide your response in the following JSON format:
{
  "review_status": {
    "missing_labels": [
      {
        "text": "exact text from letter",
        "context": "complete sentence/passage",
        "qualifier": "one of the valid categories",
        "laterality": "left/right/bilateral/NA",
        "presence": "confirmed/suspected/resolved/negated/NA",
        "primary_secondary": "primary/secondary/NA",
        "experiencer": "Patient/Family/NA",
        "treatment_stage": "pre-treatment/post-treatment/NA",
        "snomed_ct": "code or NA"
      }
    ],
    "corrections": {
      "label_text": {
        "field_to_correct": "new_value",
        "reason": "brief explanation"
      }
    },
    "deletions": [
      {
        "label_text": "text to remove",
        "reason": "brief explanation"
      }
    ]
  }
}

If no changes are needed, respond with:
{
  "review_status": {
    "missing_labels": [],
    "corrections": {},
    "deletions": []
  }
}

Original Clinical Letter:
'''

def create_review_label_prompt(formatted_labels_json, formatted_letter):
    REVIEW_LABELS_PROMPT = REVIEW_LABELS_PROMPT_INTRO+f'''
{formatted_labels_json}
'''+REVIEW_LABELS_PROMPT_MAIN+f'''

{formatted_letter}

Provide only the JSON response without any additional text.
'''

    return REVIEW_LABELS_PROMPT

# Step 4: Get modification/additions
operation_type = 'None' # Initialise necessary variables
label_text = 'None'
reason = 'None'
current_values_json = 'None'
existing_labels_summary = 'None'

def create_modify_add_prompt(operation_type, label_text, reason, current_values_json, existing_labels_summary, formatted_letter):
    MODIFY_OR_ADD_LABEL_PROMPT = f'''
Update or add the following label based on the clinical letter. Generate a complete JSON entry following the same format as the original extraction.

Operation Type: {operation_type}  # "modify" or "add"
Label Text: {label_text}
Reason for Change: {reason}

For modification, current values are:
{current_values_json}

Clinical Letter Context:
{formatted_letter}

Provide a single JSON entry for this label using this exact format:'''+'''
{
  "text": "'''+f'{label_text}'+'''",
  "context": "complete sentence/passage containing the label",
  "qualifier": "MUST BE ONE OF: Sociodemographics, Signs, Symptoms, Diagnoses, Treatments, Risk Factors, Test Results",
  "laterality": "MUST BE ONE OF: left, right, bilateral, NA",
  "presence": "MUST BE ONE OF: confirmed, suspected, resolved, negated, NA",
  "primary_secondary": "MUST BE ONE OF: primary, secondary, NA",
  "experiencer": "MUST BE ONE OF: Patient, Family, NA",
  "treatment_stage": "MUST BE ONE OF: pre-treatment, post-treatment, NA",
  "snomed_ct": "code or NA"
}

Requirements:
1. Use exact text from the letter
2. Provide complete context sentence
3. Only use specified valid values for each field
4. Use "NA" for any field that doesn't apply
5. Ensure consistency with other labels
6. Consider temporal and clinical relationships

Original Extraction Context:'''+f'''
{existing_labels_summary}

Provide only the JSON entry without any additional text or explanation.
'''
    return MODIFY_OR_ADD_LABEL_PROMPT

## Define classes and functions

In [ ]:
class LLMOutputParser:
    """Parser for extracting and validating JSON from LLM outputs."""

    def __init__(self):
        self.logger = logging.getLogger(__name__)

        # Define expected fields and their types
        self.expected_fields = {
            "context": str,
            "qualifier": str,
            "laterality": str,
            "presence": str,
            "primary_secondary": str,
            "experiencer": str,
            "treatment_stage": str,
            "snomed_ct": str
        }

        # Valid values for categorical fields
        self.valid_values = {
            "qualifier": {"Sociodemographics", "Signs", "Symptoms", "Diagnoses",
                         "Treatments", "Risk Factors", "Test Results"},
            "laterality": {"left", "right", "bilateral", "NA"},
            "presence": {"confirmed", "suspected", "resolved", "negated", "NA"},
            "primary_secondary": {"primary", "secondary", "NA"},
            "experiencer": {"Patient", "Family", "NA"},
            "treatment_stage": {"pre-treatment", "post-treatment", "NA"}
        }

    def extract_json_from_text(self, text: str) -> Optional[str]:
        """Extract JSON string from LLM output text."""
        try:
            # Find text between first { and last }
            json_match = re.search(r'\{.*\}', text, re.DOTALL)
            if json_match:
                return json_match.group()
            return None
        except Exception as e:
            self.logger.error(f"Error extracting JSON: {str(e)}")
            return None

    def validate_label_entry(self, label: str, entry: Dict[str, Any]) -> Dict[str, Any]:
        """Validate and clean a single label entry."""
        cleaned_entry = {}

        # Check for required fields
        for field, expected_type in self.expected_fields.items():
            value = entry.get(field)

            # Handle missing fields
            if value is None:
                self.logger.warning(f"Missing field '{field}' for label '{label}'. Setting to 'NA'")
                cleaned_entry[field] = "NA"
                continue

            # Validate and clean value
            if field in self.valid_values:
                value = str(value).lower()
                if value not in {v.lower() for v in self.valid_values[field]}:
                    self.logger.warning(
                        f"Invalid value '{value}' for field '{field}' in label '{label}'. Setting to 'NA'"
                    )
                    value = "NA"

            cleaned_entry[field] = value

        return cleaned_entry

    def parse_llm_output(self, llm_output: str) -> Dict[str, Dict[str, Any]]:
        """Parse and validate LLM output into structured format."""
        try:
            # Extract JSON from text if needed
            json_str = self.extract_json_from_text(llm_output)
            if not json_str:
                raise ValueError("No JSON found in output")

            # Parse JSON
            parsed = json.loads(json_str)

            # Extract labels dictionary
            labels = parsed.get("labels", {})
            if not labels:
                raise ValueError("No labels found in JSON")

            # Validate and clean each label entry
            cleaned_labels = {}
            for label, entry in labels.items():
                try:
                    cleaned_labels[label] = self.validate_label_entry(label, entry)
                except Exception as e:
                    self.logger.error(f"Error processing label '{label}': {str(e)}")
                    continue

            return cleaned_labels

        except json.JSONDecodeError as e:
            self.logger.error(f"JSON parsing error: {str(e)}")
            raise
        except Exception as e:
            self.logger.error(f"Error parsing LLM output: {str(e)}")
            raise

    def to_dataframe(self, parsed_json):
        """
        Convert parsed JSON from LLM output to a pandas DataFrame

        Args:
            parsed_json (dict): Dictionary containing extracted labels and attributes

        Returns:
            pd.DataFrame: DataFrame with labels and metadata
        """
        records = [
            {
                'text': text,
                **attributes
            }
            for text, attributes in parsed_json.items()
        ]

        df = pd.DataFrame(records)
        df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
        df = df.replace(['na', 'NA'], None)

        return df

In [ ]:
class ChatGPTInstance:
    """Instance of ChatGPT for clinical letter analysis with preprocessing capability."""

    def __init__(self, api_key, model="gpt-3.5-turbo", temperature=0, seed=42,
                 output_dir="outputs", letter_id=None):
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        self.temperature = temperature
        self.seed = seed
        self.messages = []
        self.parser = LLMOutputParser()

        # Setup output management
        self.letter_id = letter_id or str(uuid.uuid4())[:8]
        self.output_dir = Path(output_dir) / self.letter_id
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # Setup logging
        self.setup_logging()

        # Initialize run data
        self.run_data = {
            "raw_letter": None,
            "preprocessed_letter": None,
            "initial_extraction": None,
            "review": None,
            "final_extraction": None,
            "final_dataframe": None,
            "run_metadata": {
                "model": model,
                "temperature": temperature,
                "seed": seed,
                "timestamp": datetime.now().isoformat(),
                "letter_id": self.letter_id
            }
        }

    def setup_logging(self):
            """Setup logging configuration."""
            log_file = self.output_dir / "extraction_run.log"

            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s',
                handlers=[
                    logging.FileHandler(log_file),
                    logging.StreamHandler()
                ]
            )
            self.logger = logging.getLogger(f"ChatGPT_{self.letter_id}")

    def save_conversation_log(self):
        """Save the complete conversation history."""
        conversation_file = self.output_dir / "conversation_log.json"
        with open(conversation_file, "w") as f:
            json.dump(self.messages, f, indent=2)

    def save_json_output(self, data, stage):
        """Save JSON output for a specific stage."""
        if data is None:
            data = {"labels": {}}  # Empty JSON structure
            self.logger.warning(f"Empty JSON created for {stage}")

        output_file = self.output_dir / f"{stage}.json"
        with open(output_file, "w") as f:
            json.dump(data, f, indent=2)

    def save_dataframe(self, df):
        """Save DataFrame in multiple formats."""
        base_path = self.output_dir / "final_output"
        df.to_csv(f"{base_path}.csv", index=False)
        df.to_excel(f"{base_path}.xlsx", index=False)

    @staticmethod
    def preprocess_clinical_letter(letter_json: dict) -> str:
        """Preprocess clinical letter JSON into formatted string."""
        exclude_keys = {'url', 'medical specialty', 'sample name', 'keywords'}

        sections = []
        for key, value in letter_json.items():
            if key not in exclude_keys:
                display_title = key.replace('_', ' ').title()
                if value and str(value).strip():
                    sections.append(f"{display_title}:\n{value}\n")

        return "\n".join(sections)

    def load_letter(self, file_path: str | Path) -> str:
        """
        Load and preprocess a clinical letter from JSON file.

        Args:
            file_path: Path to the JSON file

        Returns:
            str: Preprocessed letter text
        """
        file_path = Path(file_path)
        self.logger.info(f"Loading letter from {file_path}")

        try:
            with open(file_path, 'r') as f:
                letter_json = json.load(f)

            # Store raw letter
            self.run_data["raw_letter"] = letter_json

            # Preprocess letter
            preprocessed_letter = self.preprocess_clinical_letter(letter_json)
            self.run_data["preprocessed_letter"] = preprocessed_letter

            # Save both raw and preprocessed versions
            with open(self.output_dir / "raw_letter.json", 'w') as f:
                json.dump(letter_json, f, indent=2)

            with open(self.output_dir / "preprocessed_letter.txt", 'w') as f:
                f.write(preprocessed_letter)

            self.logger.info("Letter successfully loaded and preprocessed")
            return preprocessed_letter

        except Exception as e:
            self.logger.error(f"Error loading letter: {str(e)}")
            raise

    def set_pre_prompt(self, pre_prompt):
        """Sets the initial system message (pre-prompt) for the conversation."""
        self.messages = [{"role": "system", "content": pre_prompt}]

    def prompt(self, message, validate_json=False):
            """
            Enhanced prompt with logging.

            Args:
                message (str): The prompt message
                validate_json (bool): Whether to validate JSON response

            Returns:
                str or dict: Raw response or parsed JSON if validation requested
            """
            self.logger.info(f"Sending prompt: {message[:100]}...")
            self.messages.append({"role": "user", "content": message})

            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=self.messages,
                    temperature=self.temperature,
                    seed=self.seed
                )
                assistant_message = response.choices[0].message.content
                self.messages.append({"role": "assistant", "content": assistant_message})

                self.logger.info(f"Received response: {assistant_message[:100]}...")

                if validate_json:
                    try:
                        parsed = self.parser.parse_llm_output(assistant_message)
                        self.logger.info("Successfully parsed JSON response")
                        return parsed
                    except Exception as e:
                        self.logger.error(f"JSON validation failed: {str(e)}")
                        return self._retry_json_extraction(assistant_message)

                return assistant_message

            except Exception as e:
                self.logger.error(f"Prompt failed: {str(e)}")
                raise

    def _retry_json_extraction(self, failed_response, max_retries=1):
        """
        Attempts to fix invalid JSON responses.

        Args:
            failed_response (str): The response that failed validation
            max_retries (int): Maximum number of retry attempts
        """
        retry_prompt = f'''
        The previous response was not in valid JSON format.
        Please reformat the following content as a valid JSON object
        following the specified structure exactly:

        {failed_response}
        '''

        for _ in range(max_retries):
            try:
                new_response = self.prompt(retry_prompt)
                return self.parser.parse_llm_output(new_response)
            except Exception as e:
                logging.error(f"Retry failed: {str(e)}")

        raise ValueError("Failed to obtain valid JSON after retries")

    def analyze_clinical_letter(self, letter_dict):
        """Enhanced analysis with comprehensive output management."""
        self.logger.info(f"Starting analysis for letter {self.letter_id}")

        try:
            # Initial extraction
            self.logger.info("Performing initial extraction")
            formatted_letter = json.dumps(letter_dict, indent=2)
            LETTER_PROMPT = create_letter_prompt(formatted_letter)
            initial_extraction = self.prompt(
                LETTER_PROMPT,
                validate_json=True
            )
            self.run_data["initial_extraction"] = initial_extraction
            self.save_json_output(initial_extraction, "initial_extraction")


            # Create final DataFrame
            self.logger.info("Creating final DataFrame")
            print(initial_extraction)
            final_df = self.parser.to_dataframe(initial_extraction)
            self.run_data["final_dataframe"] = final_df
            self.save_dataframe(final_df)

            # Save conversation log
            self.save_conversation_log()

            return self.run_data

        except Exception as e:
            self.logger.error(f"Analysis failed: {str(e)}")
            raise
        finally:
            # Save run metadata
            with open(self.output_dir / "run_metadata.json", "w") as f:
                json.dump(self.run_data["run_metadata"], f, indent=2)

    def _process_review_changes(self, original_extraction, review, letter_dict):
        """
        Processes changes suggested by the review.

        Args:
            original_extraction (dict): Original label extraction
            review (dict): Review response
            letter_dict (dict): Original letter
        """
        # Handle modifications
        for label_text, changes in review["review_status"]["corrections"].items():
            vars = init_modify_add_label_vars(
                operation="modify",
                label_text=label_text,
                reason=changes.get("reason", "Review correction"),
                current_values_json=original_extraction,
                clinical_letter=letter_dict
            )
            MODIFY_OR_ADD_LABEL_PROMPT = create_modify_add_prompt(**vars)
            modified_label = self.prompt(
                MODIFY_OR_ADD_LABEL_PROMPT,
                validate_json=True
            )
            original_extraction["labels"][label_text].update(modified_label)

        # Handle additions
        for new_label in review["review_status"]["missing_labels"]:
            vars = init_modify_add_label_vars(
                operation="add",
                label_text=new_label["text"],
                reason="Review addition",
                original_extraction=original_extraction,
                clinical_letter=letter_dict
            )
            MODIFY_OR_ADD_LABEL_PROMPT = create_modify_add_prompt(**vars)
            added_label = self.prompt(
                MODIFY_OR_ADD_LABEL_PROMPT,
                validate_json=True
            )
            original_extraction["labels"][new_label["text"]] = added_label

        return original_extraction

    def get_last_response(self):
        """Returns the last response from the assistant."""
        if self.messages and self.messages[-1]["role"] == "assistant":
            return self.messages[-1]["content"]
        return None

    def reset(self):
        """Resets the conversation, clearing all context."""
        self.messages = []

## Example run

In [ ]:
# Initialize with output management
gpt = ChatGPTInstance(
    api_key='replace_with_your_key',
    model="gpt-4o-2024-08-06",
    temperature=0,
    output_dir="../results/pilot_cot/openai/4o/",
    letter_id="letter_0018"
)

# Run analysis
try:
    run_data = gpt.analyze_clinical_letter('../data/mt_samples/letters/letter_0018.json')

    # Access outputs including raw and preprocessed letter
    raw_letter = run_data["raw_letter"]
    preprocessed_letter = run_data["preprocessed_letter"]
    initial_json = run_data["initial_extraction"]
    review_json = run_data["review"]
    final_json = run_data["final_extraction"]
    final_df = run_data["final_dataframe"]

except Exception as e:
    print(f"Analysis failed: {str(e)}")

2024-11-19 15:20:13,928 - INFO - Starting analysis for letter letter_0018
2024-11-19 15:20:13,929 - INFO - Performing initial extraction
2024-11-19 15:20:13,929 - INFO - Sending prompt: 
Analyze the following clinical letter and extract all relevant information. Your response must be a...
2024-11-19 15:20:23,541 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-19 15:20:23,757 - INFO - Received response: ```json
{
  "labels": {
    "15-year-old": {
      "context": "The patient is a 15-year-old male who...
2024-11-19 15:20:23,758 - INFO - Successfully parsed JSON response
2024-11-19 15:20:23,761 - INFO - Creating final DataFrame


{'15-year-old': {'context': 'The patient is a 15-year-old male who presented with a persistent cough.', 'qualifier': 'sociodemographics', 'laterality': 'na', 'presence': 'confirmed', 'primary_secondary': 'na', 'experiencer': 'patient', 'treatment_stage': 'na', 'snomed_ct': 'NA'}, 'persistent cough': {'context': 'The patient is a 15-year-old male who presented with a persistent cough.', 'qualifier': 'symptoms', 'laterality': 'na', 'presence': 'confirmed', 'primary_secondary': 'primary', 'experiencer': 'patient', 'treatment_stage': 'na', 'snomed_ct': '11833005'}, 'chest X-ray': {'context': 'A chest X-ray was performed, which showed no abnormalities.', 'qualifier': 'test results', 'laterality': 'na', 'presence': 'negated', 'primary_secondary': 'na', 'experiencer': 'patient', 'treatment_stage': 'na', 'snomed_ct': '168731006'}, 'no abnormalities': {'context': 'A chest X-ray was performed, which showed no abnormalities.', 'qualifier': 'test results', 'laterality': 'na', 'presence': 'negated'

In [ ]:
import json
from IPython.display import display, Markdown

def format_chat_messages(messages, print_markdown=False):
    """
    Format chat messages as markdown and optionally display in notebook

    Args:
        messages (list): List of message dictionaries with 'role' and 'content' keys
        print_markdown (bool): Whether to display formatted markdown in notebook

    Returns:
        str: Formatted markdown string
    """
    def parse_messages(msg_str):
        try:
            return json.loads(msg_str)
        except json.JSONDecodeError:
            return msg_str

    def format_content(content):
        if isinstance(content, str):
            return content.strip()
        return json.dumps(content, indent=2)

    markdown_str = ""

    for msg in messages:
        role = msg['role'].capitalize()
        content = parse_messages(msg['content'])
        formatted_content = format_content(content)

        markdown_str += f"**{role}**: \n```\n{formatted_content}\n```\n\n"

    if print_markdown:
        display(Markdown(markdown_str))

    return markdown_str

In [54]:
formatted = format_chat_messages(gpt.messages, print_markdown=True)

**User**: 
```
Analyze the following clinical letter and extract all relevant information. Your response must be a valid JSON object matching the following specification exactly.

Required Format:
{
  "labels": {
    "exact_text": {
      "context": "string with complete sentence/passage",
      "qualifier": "MUST BE ONE OF: Sociodemographics, Signs, Symptoms, Diagnoses, Treatments, Risk Factors, Test Results",
      "laterality": "MUST BE ONE OF: left, right, bilateral, NA",
      "presence": "MUST BE ONE OF: confirmed, suspected, resolved, negated, NA",
      "primary_secondary": "MUST BE ONE OF: primary, secondary, NA",
      "experiencer": "MUST BE ONE OF: Patient, Family, NA",
      "treatment_stage": "MUST BE ONE OF: pre-treatment, post-treatment, NA",
      "snomed_ct": "string of numbers or NA"
    }
  }
}

Critical Requirements:
1. Response must be a single JSON object
2. All field names must be exactly as shown above
3. Every label must have ALL fields specified (use "NA" if not applicable)
4. No additional fields or nested objects are allowed
5. All string values must be properly escaped
6. Fields must use ONLY the enumerated values where specified
7. Context must contain complete sentences, properly escaped
8. Do not include any explanatory text before or after the JSON

Extract all instances of:
- Socio-demographics
- Signs
- Symptoms
- Diagnoses
- Treatments
- Risk Factors
- Test Results

Clinical Letter:


None

Respond only with the JSON object following the specified format.
```

**Assistant**: 
```
{
  "labels": {
    "exact_text": {
      "context": "None",
      "qualifier": "NA",
      "laterality": "NA",
      "presence": "NA",
      "primary_secondary": "NA",
      "experiencer": "NA",
      "treatment_stage": "NA",
      "snomed_ct": "NA"
    }
  }
}
```

**User**: 
```
Review the following extracted labels from a clinical letter. Verify their accuracy, completeness, and consistency. The current extraction is:


None

Review Tasks:
1. MISSING INFORMATION
   Check for any missing labels in these categories:
   - Socio-demographics
   - Signs
   - Symptoms
   - Diagnoses
   - Treatments
   - Risk Factors
   - Test Results

2. VALIDATION
   For each existing label, verify:
   - Context matches the label
   - Qualifier category is appropriate
   - Presence status is accurate
   - Primary/Secondary classification is correct
   - Experiencer assignment is accurate
   - Treatment stage is appropriate

3. CLINICAL CONSISTENCY
   Check that:
   - Related symptoms/signs are consistently labeled
   - Temporal relationships are correctly reflected
   - Family history is properly distinguished
   - Diagnostic relationships are accurate

Provide your response in the following JSON format:
{
  "review_status": {
    "missing_labels": [
      {
        "text": "exact text from letter",
        "context": "complete sentence/passage",
        "qualifier": "one of the valid categories",
        "laterality": "left/right/bilateral/NA",
        "presence": "confirmed/suspected/resolved/negated/NA",
        "primary_secondary": "primary/secondary/NA",
        "experiencer": "Patient/Family/NA",
        "treatment_stage": "pre-treatment/post-treatment/NA",
        "snomed_ct": "code or NA"
      }
    ],
    "corrections": {
      "label_text": {
        "field_to_correct": "new_value",
        "reason": "brief explanation"
      }
    },
    "deletions": [
      {
        "label_text": "text to remove",
        "reason": "brief explanation"
      }
    ]
  }
}

If no changes are needed, respond with:
{
  "review_status": {
    "missing_labels": [],
    "corrections": {},
    "deletions": []
  }
}

Original Clinical Letter:


None

Provide only the JSON response without any additional text.
```

**Assistant**: 
```
{
  "review_status": {
    "missing_labels": [
      {
        "text": "None",
        "context": "None",
        "qualifier": "NA",
        "laterality": "NA",
        "presence": "NA",
        "primary_secondary": "NA",
        "experiencer": "NA",
        "treatment_stage": "NA",
        "snomed_ct": "NA"
      }
    ],
    "corrections": {},
    "deletions": []
  }
}
```

**User**: 
```
The previous response was not in valid JSON format. 
        Please reformat the following content as a valid JSON object 
        following the specified structure exactly:

        {
  "review_status": {
    "missing_labels": [
      {
        "text": "None",
        "context": "None",
        "qualifier": "NA",
        "laterality": "NA",
        "presence": "NA",
        "primary_secondary": "NA",
        "experiencer": "NA",
        "treatment_stage": "NA",
        "snomed_ct": "NA"
      }
    ],
    "corrections": {},
    "deletions": []
  }
}
```

**Assistant**: 
```
{
  "review_status": {
    "missing_labels": [
      {
        "text": "None",
        "context": "None",
        "qualifier": "NA",
        "laterality": "NA",
        "presence": "NA",
        "primary_secondary": "NA",
        "experiencer": "NA",
        "treatment_stage": "NA",
        "snomed_ct": "NA"
      }
    ],
    "corrections": {},
    "deletions": []
  }
}
```

